# Laboratorio: Comparación T5 vs FLAN-T5

## SECCIÓN 1: Setup y Carga de Datos

Esta sección está completa. Solo ejecuta las celdas.

In [ ]:
# Instalación de dependencias
!pip install transformers datasets evaluate accelerate -q
!pip install torch -q
!pip install scikit-learn -q

# Imports necesarios
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import numpy as np
import evaluate
from datetime import datetime

print("✓ Dependencias cargadas")

In [ ]:
# Cargar datasets con subsets fijos (5000 train, 1000 validation cada uno)
print("Cargando datasets...\n")

# Dataset 1: SST-2 (Sentimiento de películas)
print("  - SST-2 (Sentimiento de películas)...")
dataset_sst2 = load_dataset("glue", "sst2")
dataset_sst2_train = dataset_sst2["train"].select(range(5000))
dataset_sst2_val = dataset_sst2["validation"].select(range(872)) # No tiene mas ejemplos

# Dataset 2: Amazon Polarity (Reviews de productos)
print("  - Amazon Polarity (Reviews de productos)...")
dataset_amazon = load_dataset("amazon_polarity")
dataset_amazon_train = dataset_amazon["train"].select(range(5000))
dataset_amazon_val = dataset_amazon["test"].select(range(1000))

# Dataset 3: AG News (Clasificación de noticias en 4 categorías)
print("  - AG News (Noticias en 4 categorías)...")
dataset_agnews = load_dataset("ag_news")
dataset_agnews_train = dataset_agnews["train"].select(range(5000))
dataset_agnews_val = dataset_agnews["test"].select(range(1000))

print("\n✓ Datasets cargados")

In [ ]:
# Exploración de los datasets
print("--- SST-2 (Sentimiento de películas) ---")
print(f"Train: {len(dataset_sst2_train)} ejemplos")
print(f"Validation: {len(dataset_sst2_val)} ejemplos")
print(f"Clases: 0=negative, 1=positive")
print(f"Ejemplo: {dataset_sst2_train[0]}")

print("\n--- Amazon Polarity (Reviews de productos) ---")
print(f"Train: {len(dataset_amazon_train)} ejemplos")
print(f"Validation: {len(dataset_amazon_val)} ejemplos")
print(f"Clases: 0=negative, 1=positive")
print(f"Ejemplo: {dataset_amazon_train[0]}")

print("\n--- AG News (Noticias) ---")
print(f"Train: {len(dataset_agnews_train)} ejemplos")
print(f"Validation: {len(dataset_agnews_val)} ejemplos")
print(f"Clases: 0=World, 1=Sports, 2=Business, 3=Sci/Tech")
print(f"Ejemplo: {dataset_agnews_train[0]}")

In [ ]:
# Cargar tokenizador
model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print("✓ Tokenizador T5-base cargado")

## SECCIÓN 2: Preprocesamiento de Datos

En esta sección verás un ejemplo completo (SST-2) y tendrás que adaptar el código para los otros dos datasets.

### Ejemplo Completo: SST-2

Usa este código como referencia para completar los TODOs siguientes.

In [ ]:
def preprocess_function_sst2(examples):
    """
    Preprocesa el dataset SST-2 para T5.
    Convierte la tarea de clasificación a formato text-to-text.
    """
    # Prefijo que indica la tarea
    prefix = "sst2 sentence: "

    # Mapeo de etiquetas numéricas a texto
    label_map = {0: "negative", 1: "positive"}

    # Añadir prefijo a cada oración
    inputs = [prefix + doc for doc in examples["sentence"]]

    # Tokenizar inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding=False)

    # Convertir labels a texto
    labels_text = [label_map[label] for label in examples["label"]]

    # Tokenizar labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(labels_text, max_length=2, truncation=True, padding=False)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Aplicar preprocesamiento
tokenized_sst2_train = dataset_sst2_train.map(
    preprocess_function_sst2,
    batched=True,
    remove_columns=dataset_sst2_train.column_names
)
tokenized_sst2_val = dataset_sst2_val.map(
    preprocess_function_sst2,
    batched=True,
    remove_columns=dataset_sst2_val.column_names
)

print("✓ SST-2 preprocesado")

### TODO 1: Preprocesar Amazon Polarity

**Pistas:**
- El dataset usa `'content'` y `'title'` en vez de `'sentence'`
- Puedes concatenar: `title + ' ' + content`
- El prefijo debe ser: `'amazon review: '`
- Las labels son iguales: `0=negative, 1=positive`

In [ ]:
def preprocess_function_amazon(examples):
    """
    TODO: Completa esta función basándote en el ejemplo de SST-2
    """
    # TODO: Define el prefijo
    prefix = "amazon review: "

    # TODO: Define el mapeo de labels (igual que SST-2)
    label_map = {0: "negative", 1: "positive"}

    # TODO: Concatena title + content y añade prefijo
    # Hint: inputs = [prefix + title + " " + content for title, content in zip(...)]
    inputs = # TU CÓDIGO AQUÍ

    # TODO: Tokeniza los inputs (copia de SST-2)
    model_inputs = # TU CÓDIGO AQUÍ

    # TODO: Convierte labels a texto
    labels_text = # TU CÓDIGO AQUÍ

    # TODO: Tokeniza las labels
    with tokenizer.as_target_tokenizer():
        labels = # TU CÓDIGO AQUÍ

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# TODO: Aplica el preprocesamiento (copia la estructura de SST-2)
tokenized_amazon_train = # TU CÓDIGO AQUÍ
tokenized_amazon_val = # TU CÓDIGO AQUÍ

print("✓ Amazon Polarity preprocesado")

### TODO 2: Preprocesar AG News

**Pistas:**
- El dataset usa `'text'` para el contenido
- El prefijo debe ser: `'ag news: '`
- Ahora son 4 clases: `0=World, 1=Sports, 2=Business, 3=Sci/Tech`
- `max_length` para labels debe ser 3 (palabras más largas)

In [ ]:
def preprocess_function_agnews(examples):
    """
    TODO: Completa esta función para AG News (4 clases)
    """
    # TODO: Define el prefijo
    prefix = # TU CÓDIGO AQUÍ

    # TODO: Define el mapeo de labels (4 clases ahora)
    label_map = {
        0: # TU CÓDIGO AQUÍ,
        1: # TU CÓDIGO AQUÍ,
        2: # TU CÓDIGO AQUÍ,
        3: # TU CÓDIGO AQUÍ
    }

    # TODO: Añade prefijo al texto
    inputs = # TU CÓDIGO AQUÍ

    # TODO: Tokeniza los inputs
    model_inputs = # TU CÓDIGO AQUÍ

    # TODO: Convierte labels a texto
    labels_text = # TU CÓDIGO AQUÍ

    # TODO: Tokeniza las labels (max_length=3 porque las palabras son más largas)
    with tokenizer.as_target_tokenizer():
        labels = # TU CÓDIGO AQUÍ

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# TODO: Aplica el preprocesamiento
tokenized_agnews_train = # TU CÓDIGO AQUÍ
tokenized_agnews_val = # TU CÓDIGO AQUÍ

print("✓ AG News preprocesado")

## SECCIÓN 3: Fine-tuning de T5-base

Esta sección está completa. El código entrenará T5-base en los 3 datasets automáticamente.

In [ ]:
# Métricas
metric_accuracy = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    """
    Calcula accuracy y F1 score para las predicciones.
    CORRECCIÓN: Compara strings directamente sin usar la librería evaluate para accuracy
    """
    predictions, labels = eval_pred

    # Decodificar predicciones
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Decodificar labels - reemplazar -100 con pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # CORRECCIÓN: Calcular accuracy manualmente comparando strings
    # Normalizar strings (strip y lowercase)
    decoded_preds_normalized = [pred.strip().lower() for pred in decoded_preds]
    decoded_labels_normalized = [label.strip().lower() for label in decoded_labels]

    # Calcular accuracy manualmente
    correct = sum(p == l for p, l in zip(decoded_preds_normalized, decoded_labels_normalized))
    accuracy = correct / len(decoded_labels_normalized)

    # Para F1: convertir strings a IDs numéricos
    unique_labels = sorted(list(set(decoded_labels_normalized)))
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}

    pred_ids = [label_to_id.get(pred, -1) for pred in decoded_preds_normalized]
    label_ids = [label_to_id.get(label, -1) for label in decoded_labels_normalized]

    # Filtrar predicciones inválidas (que no matchean ninguna label conocida)
    valid_indices = [i for i, pred_id in enumerate(pred_ids) if pred_id != -1]

    if len(valid_indices) > 0:
        pred_ids_valid = [pred_ids[i] for i in valid_indices]
        label_ids_valid = [label_ids[i] for i in valid_indices]

        f1 = metric_f1.compute(
            predictions=pred_ids_valid,
            references=label_ids_valid,
            average='macro'
        )
    else:
        f1 = {"f1": 0.0}

    return {
        "accuracy": accuracy,
        "f1": f1["f1"]
    }

In [ ]:
def train_t5_on_dataset(train_dataset, val_dataset, output_dir, dataset_name):
    """
    Entrena T5-base en un dataset específico.
    """
    print(f"\n🚀 Entrenando T5-base en {dataset_name}...")

    # Cargar modelo fresco
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

    # Configurar argumentos de entrenamiento
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        eval_strategy="epoch",
        learning_rate=3e-4,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        save_total_limit=1,
        num_train_epochs=1,  # Solo 1 época para velocidad
        predict_with_generate=True,
        report_to="none",
        fp16=False,
        logging_steps=100,
    )

    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        label_pad_token_id=-100
    )

    # Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Entrenar
    start_time = datetime.now()
    trainer.train()
    end_time = datetime.now()

    # Evaluar
    eval_results = trainer.evaluate()

    print(f"✓ Entrenamiento completado en {(end_time - start_time).seconds}s")
    print(f"  Accuracy: {eval_results['eval_accuracy']:.4f}")
    print(f"  F1 Score: {eval_results['eval_f1']:.4f}")

    return trainer, eval_results

In [ ]:
# Entrenar en los 3 datasets (esto tomará ~15-20 minutos)
print("Entrenando T5-base en los 3 datasets...\n")

trainer_sst2, results_sst2 = train_t5_on_dataset(
    tokenized_sst2_train,
    tokenized_sst2_val,
    "t5-sst2-finetuned",
    "SST-2"
)

trainer_amazon, results_amazon = train_t5_on_dataset(
    tokenized_amazon_train,
    tokenized_amazon_val,
    "t5-amazon-finetuned",
    "Amazon Polarity"
)

trainer_agnews, results_agnews = train_t5_on_dataset(
    tokenized_agnews_train,
    tokenized_agnews_val,
    "t5-agnews-finetuned",
    "AG News"
)

print("\n✓ Fine-tuning de T5-base completo para los 3 datasets")

## SECCIÓN 4: Evaluación FLAN-T5 Zero-Shot

Ahora diseñarás prompts para FLAN-T5 y verás cómo se desempeña sin entrenamiento.

In [ ]:
# Cargar FLAN-T5
print("Cargando FLAN-T5...")
model_flan = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer_flan = AutoTokenizer.from_pretrained("google/flan-t5-base")
print("✓ FLAN-T5 cargado")

In [ ]:
def evaluate_flan_t5(dataset, prompt_template, label_map):
    """
    Evalúa FLAN-T5 en zero-shot sobre un dataset.
    CORRECCIÓN: Calcula accuracy manualmente sin usar metric_accuracy.compute()
    """
    predictions = []
    references = []

    print(f"  Evaluando {len(dataset)} ejemplos...")

    for i, example in enumerate(dataset):
        if i % 200 == 0:
            print(f"    Progreso: {i}/{len(dataset)}")

        # Generar prompt
        prompt = prompt_template(example)

        # Generar predicción
        input_ids = tokenizer_flan(prompt, return_tensors="pt", max_length=512, truncation=True).input_ids
        outputs = model_flan.generate(input_ids, max_length=10)
        prediction = tokenizer_flan.decode(outputs[0], skip_special_tokens=True)

        predictions.append(prediction)
        references.append(label_map[example["label"]])

    # Normalizar strings (strip y lowercase)
    predictions_normalized = [pred.strip().lower() for pred in predictions]
    references_normalized = [ref.strip().lower() for ref in references]

    # Calcular accuracy manualmente
    correct = sum(p == r for p, r in zip(predictions_normalized, references_normalized))
    accuracy = correct / len(references_normalized)

    # F1 score - convertir strings a IDs numéricos
    unique_labels = sorted(list(set(references_normalized)))
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}

    pred_ids = [label_to_id.get(pred, -1) for pred in predictions_normalized]
    ref_ids = [label_to_id.get(ref, -1) for ref in references_normalized]

    # Filtrar predicciones inválidas
    valid_indices = [i for i, pred_id in enumerate(pred_ids) if pred_id != -1]

    if len(valid_indices) > 0:
        pred_ids_valid = [pred_ids[i] for i in valid_indices]
        ref_ids_valid = [ref_ids[i] for i in valid_indices]

        f1 = metric_f1.compute(
            predictions=pred_ids_valid,
            references=ref_ids_valid,
            average='macro'
        )
    else:
        f1 = {"f1": 0.0}

    return {
        "accuracy": accuracy,
        "f1": f1["f1"]
    }

### TODO 3: Diseña prompts efectivos para FLAN-T5

Experimenta con diferentes formulaciones y observa cuál funciona mejor.

**Características de un buen prompt:**
- Instrucción clara y directa
- Especifica las opciones exactas de respuesta
- Contexto mínimo pero suficiente

In [ ]:
# TODO: Diseña un prompt para SST-2
def prompt_sst2(example):
    """
    TODO: Diseña un prompt claro para clasificación de sentimiento.
    El prompt debe pedirle a FLAN-T5 que responda 'positive' o 'negative'.
    """
    prompt = f"""# TU CÓDIGO AQUÍ
    # Ejemplo: "Classify the sentiment of this review as positive or negative: {example['sentence']}"
    """
    return prompt

# TODO: Diseña un prompt para Amazon
def prompt_amazon(example):
    """
    TODO: Diseña un prompt para reviews de productos Amazon.
    """
    prompt = f"""# TU CÓDIGO AQUÍ
    """
    return prompt

# TODO: Diseña un prompt para AG News
def prompt_agnews(example):
    """
    TODO: Diseña un prompt para clasificación de noticias en 4 categorías.
    Las categorías son: World, Sports, Business, Sci/Tech
    """
    prompt = f"""# TU CÓDIGO AQUÍ
    """
    return prompt

In [ ]:
# Mapeos de labels
label_map_sst2 = {0: "negative", 1: "positive"}
label_map_amazon = {0: "negative", 1: "positive"}
label_map_agnews = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

In [ ]:
# Evaluar FLAN-T5 en los 3 datasets
print("\nEvaluando FLAN-T5 en SST-2...")
results_flan_sst2 = evaluate_flan_t5(dataset_sst2_val, prompt_sst2, label_map_sst2)
print(f"✓ SST-2 - Accuracy: {results_flan_sst2['accuracy']:.4f}, F1: {results_flan_sst2['f1']:.4f}")

print("\nEvaluando FLAN-T5 en Amazon...")
results_flan_amazon = evaluate_flan_t5(dataset_amazon_val, prompt_amazon, label_map_amazon)
print(f"✓ Amazon - Accuracy: {results_flan_amazon['accuracy']:.4f}, F1: {results_flan_amazon['f1']:.4f}")

print("\nEvaluando FLAN-T5 en AG News...")
results_flan_agnews = evaluate_flan_t5(dataset_agnews_val, prompt_agnews, label_map_agnews)
print(f"✓ AG News - Accuracy: {results_flan_agnews['accuracy']:.4f}, F1: {results_flan_agnews['f1']:.4f}")

## SECCIÓN 5: Comparación y Análisis

Ahora compararemos los resultados y discutiremos las implicaciones.

In [ ]:
# Tabla comparativa - Accuracy
print("\n📊 TABLA COMPARATIVA - ACCURACY")
print("-" * 80)
print(f"{'Dataset':<20} | {'T5 Fine-tuned':<15} | {'FLAN-T5 Zero-shot':<18} | {'Diferencia':<12}")
print("-" * 80)

diff_sst2 = results_sst2['eval_accuracy'] - results_flan_sst2['accuracy']
print(f"{'SST-2':<20} | {results_sst2['eval_accuracy']:>14.2%} | {results_flan_sst2['accuracy']:>17.2%} | {diff_sst2:>+11.2%}")

diff_amazon = results_amazon['eval_accuracy'] - results_flan_amazon['accuracy']
print(f"{'Amazon Polarity':<20} | {results_amazon['eval_accuracy']:>14.2%} | {results_flan_amazon['accuracy']:>17.2%} | {diff_amazon:>+11.2%}")

diff_agnews = results_agnews['eval_accuracy'] - results_flan_agnews['accuracy']
print(f"{'AG News':<20} | {results_agnews['eval_accuracy']:>14.2%} | {results_flan_agnews['accuracy']:>17.2%} | {diff_agnews:>+11.2%}")

print("-" * 80)

In [ ]:
# Tabla comparativa - F1 Score
print("\n📊 TABLA COMPARATIVA - F1 SCORE (MACRO)")
print("-" * 80)
print(f"{'Dataset':<20} | {'T5 Fine-tuned':<15} | {'FLAN-T5 Zero-shot':<18} | {'Diferencia':<12}")
print("-" * 80)

diff_f1_sst2 = results_sst2['eval_f1'] - results_flan_sst2['f1']
print(f"{'SST-2':<20} | {results_sst2['eval_f1']:>14.2%} | {results_flan_sst2['f1']:>17.2%} | {diff_f1_sst2:>+11.2%}")

diff_f1_amazon = results_amazon['eval_f1'] - results_flan_amazon['f1']
print(f"{'Amazon Polarity':<20} | {results_amazon['eval_f1']:>14.2%} | {results_flan_amazon['f1']:>17.2%} | {diff_f1_amazon:>+11.2%}")

diff_f1_agnews = results_agnews['eval_f1'] - results_flan_agnews['f1']
print(f"{'AG News':<20} | {results_agnews['eval_f1']:>14.2%} | {results_flan_agnews['f1']:>17.2%} | {diff_f1_agnews:>+11.2%}")

print("-" * 80)

## Preguntas de Reflexión

Discute las siguientes preguntas con tu grupo:

1. **¿En qué dataset FLAN-T5 se acerca más al rendimiento de T5 fine-tuned?**
   ¿Por qué crees que esto ocurre?

2. **¿Por qué crees que AG News (4 clases) podría ser más difícil para FLAN-T5 en zero-shot comparado con los datasets binarios?**

3. **Observa las diferencias de accuracy. ¿Cuándo vale la pena hacer fine-tuning en vez de usar FLAN-T5 zero-shot en producción?**

4. **¿Cómo afecta la calidad del prompt al rendimiento de FLAN-T5?**
   Experimenta modificando los prompts y observa si mejoran los resultados.

5. **Si tuvieras que deployar un sistema de clasificación de texto en producción:**
   - ¿Con datos etiquetados disponibles? → ¿Qué usarías?
   - ¿Sin datos etiquetados? → ¿Qué usarías?
   - ¿Para múltiples tareas diferentes? → ¿Qué usarías?

6. **BONUS: ¿Qué pasaría si usaras few-shot prompting con FLAN-T5?**
   (Añadir 2-3 ejemplos en el prompt antes de la predicción)

## ✓ Laboratorio Completado

### Resumen de aprendizajes clave:

1. **T5-base** es un modelo poderoso que requiere fine-tuning para tareas específicas
2. **FLAN-T5** demuestra el poder del instruction tuning para generalización zero-shot
3. La **calidad del prompt** es crucial para el rendimiento de FLAN-T5
4. **Fine-tuning vs Zero-shot** es una decisión que depende del contexto y recursos disponibles
5. Entender estos trade-offs es esencial para deployment en producción